<img align="center" src="../extra/logos/logos.png" width='1200px'>

# Caso de estudio 2

Este caso de estudio es parte de un ensayo para el monitoreo de humedales por medio de imágenes satelitales Landsat en la región de Atacama, Chile.

Nuestro objetivo es replicar este estudio realizando algunas ligeras modificaciones. [Aquí](https://cases.dataobservatory.net/datacube/monitoreo-humedales.html) se puede encontrar mayor información sobre el estudio.

Las actividades claves son:
* Definir área de estudio, temporalidad, sensores y bandas a utilizar
* Limpiar los datos
* Definir índices a utilizar
* Realizar compuestos
* Obtener estadísticas zonales
* Visualizar resultados

***

In [ ]:
import os

os.environ["USE_PYGEOS"] = "0"

import datacube
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from odc.ui import DcViewer
from datacube.utils import masking
from datacube.utils.rio import configure_s3_access

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

from dea_tools.plotting import display_map, rgb

In [ ]:
dc = datacube.Datacube(app='caso_humedal') 

# Definir área de estudio, temporalidad, sensores y bandas a utilizar

El área de estudio será definida momentaneamente como un rectángulo, luego lo acotaremos.

In [ ]:
study_area_lat = (-27.446, -27.327) # (ymin, ymax)
study_area_lon = (-69.067, -68.966) # (xmin, xmax)

In [ ]:
display_map(x=study_area_lon, y=study_area_lat)

Listando los productos terminados con 'c2l2_sr' podemos ver los productos de Landsat disponibles para reflectancia de la superficie.

In [ ]:
products = [f['name'] for i, f in dc.list_products().iterrows() if f['name'].endswith('c2l2_sr')]
products

In [ ]:
dc.list_measurements().loc[products]

Ahora que se tiene mayor claridad, sobre los productos, podemos especificar las bandas de interes, y revisar sus fechas. Utilizando las bandas azul, verde, roja e infrarroja podemos armar los índices de vegetación más conocidos para explorar los humedales. También requerimos las bandas de calidad, para filtrar las nubes y otros artefactos.

Aprovecharemos la mejor resolución espacial que nos entrega Landsat, que es de 30x30 metros, y utilizaremos el siguiente sistema de referencia de coordenadas: "WGS84 UTM 19 Sur (EPSG: 32719)"

Primero, configuramos la siguiente consulta:

In [ ]:
set_measurements = [
    "blue",
    "green",
    "red",
    "nir08",
    "qa_pixel",
    "qa_radsat"
]

set_crs = 'EPSG:32719'

set_resolution = (-30, 30)

Luego, cargamos los datos para cada producto

In [ ]:
images = {}
for set_product in products:
    images[set_product] = dc.load(
        product=set_product,
        x=study_area_lon,
        y=study_area_lat,
        # time=set_time,
        measurements=set_measurements,
        output_crs=set_crs,
        resolution=set_resolution,
        dask_chunks={"time": 1},
        group_by="solar_day",
        skip_broken_datasets=True
    )

y consultamos el rango de fechas con imágenes disponibles

In [ ]:
pd.DataFrame( {v: [min(images[v].time.values), max(images[v].time.values)] for v in images if images[v]} )

Seleccionamos las imágenes landsat 5 y 7, debido a que poseen compatibilidad radiométrica.

In [ ]:
desired_products = ['landsat5_c2l2_sr', 'landsat7_c2l2_sr']

ds = xr.concat([images[i] for i in desired_products], dim='time')

***

# Limpiar los datos

## Enmascarar valores no válidos y/o no requeridos


In [ ]:
bandas_reflectancia = ["blue", "green", "red", "nir08"]
quality_band = 'qa_pixel'
cloud_free_mask = masking.make_mask(ds.qa_pixel, cloud='not_high_confidence', cloud_shadow='not_high_confidence', snow='not_high_confidence', nodata=False)
mask_sat = ds.qa_radsat == 0
dsf = ds[bandas_reflectancia].where(cloud_free_mask & mask_sat) 
dsf.update(dsf.where((dsf >= 1) & (dsf <= 65455), np.nan))

Se aprecia que la obtención de 864 mediciones combinadas entre landsat 5 y landsat 7.

## Descartar escenas con insuficiente información

In [ ]:
valid_pixel_proportion = cloud_free_mask.sum(dim=("x", "y"))/(cloud_free_mask.shape[1] * cloud_free_mask.shape[2])
valid_threshold = 0.8
observations_to_keep = (valid_pixel_proportion >= valid_threshold)

In [ ]:
ds_keep = dsf.sel(time=observations_to_keep)#.compute()

In [ ]:
ds_keep

Se aprecia que la obtención de 536 mediciones combinadas entre landsat 5 y landsat 7 con al menos un 80% de píxeles válidos.

## Reescalar valores digitales a reflectancia

In [ ]:
ds_keep.update(ds_keep * 0.0000275 + -0.2)
ds_keep.update(ds_keep.where(ds_keep >= 0).where(ds_keep <= 1))
dsf = ds_keep

> **Nota importante**: cada banda del producto está en valores enteros (int16), para indicar la reflectancia. Se guardan de esta manera para disminuir el tamaño del archivo, pero antes de realizar cálculos, deben ser escalados de vuelta a valores decimales contenidos entre [0, 1]. Este factor de escalamiento es de 0.0000275 y un factor de adición de -0.2. Todo esto está especificado en la [documentación](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1618_Landsat-4-7_C2-L2-ScienceProductGuide-v4.pdf).

# Definir índices a utilizar

Los índices de vegetación son muy utilizados en el monitreo del estado de la vegetación, entre otros fenómenos de interés. Uno de los más usados para ver el estado de vigor (y uno de las más antiguos), es el NDVI que utiliza las bandas rojo e infra-rojo cercano.

Se usará este índice a modo de mostración, pero se podría utilizar cualquier otro.

In [ ]:
ndvi = (dsf['nir08'] - dsf['red']) / (dsf['nir08'] + dsf['red'])

In [ ]:
plt.figure(figsize=(10, 10))
ndvi.isel(time=0).plot(cmap="RdYlGn", vmin=-.2, vmax=.8)
plt.show()

In [ ]:
dsf[["red", "green", "blue"]].isel(time=0).to_array().plot.imshow(vmin=0,vmax=.3,figsize=(10,10))

# Set the title and axis labels
ax = plt.gca()
ax.set_xlabel('Easting (m)', fontweight='bold')
ax.set_ylabel('Northing (m)', fontweight='bold')

# Display the plot
plt.show()

In [ ]:
dsf = dsf.assign(ndvi=ndvi)
dsf

## Limitar análisis a áreas previamente definidas

Utilizaremos un archivo vectorial, del Ministerio del Medio Ambiente, que delimita los humedales para el año 2015. Utilizaremos el siguiente [sitio](https://humedaleschile.mma.gob.cl/inventario-humadales/) para descargar los datos a través de las siguientes celdas:

In [ ]:
import urllib.request
import os

if not os.path.exists("humedales_vector.zip"):
    urllib.request.urlretrieve("https://humedaleschile.mma.gob.cl/wp-content/uploads/2017/10/inventario_humedales_publico.gdb.zip", "humedales_vector.zip")

In [ ]:
import geopandas as gpd
import rasterstats as rs
import pyproj
from shapely.geometry import Polygon

In [ ]:
humedales = gpd.read_file('zip://humedales_vector.zip!Inventario_humedales_publico.gdb', layer='inventario_plataforma')

In [ ]:
y0, y1 = study_area_lat
x1, x0 = study_area_lon
polygon = Polygon([(x0, y0), (x0, y1), (x1, y1), (x1, y0), (x0, y0)])
clipper = gpd.GeoDataFrame([1], geometry=[polygon], crs=pyproj.CRS("EPSG:4326"))
clipper = clipper.to_crs(humedales.crs)
# clipper.plot()
humedales_ = gpd.clip(humedales, clipper)
humedales_ = humedales_.loc[humedales_.Clase != "Rios"]

Más información sobre [como armar geometrías y como cortar geometrías](https://geopandas.org/gallery/plot_clip.html). Un vistazo al archivo vectorial:

In [ ]:
humedales_.plot()

In [ ]:
humedales_or = humedales_.copy()

In [ ]:
# humedales_['geometry'] = humedales_.geometry.buffer(distance = 100) #### Not working
# humedales_ = humedales_.geometry[~(humedales_.geometry.is_empty | humedales_.geometry.isna())] #### Did not fix the problem
for i in range(humedales_.shape[0]):
    humedales_['geometry'].iloc[i] = humedales_.geometry.iloc[i].buffer(distance = 100)
# El buffer en la totalidad o en un subconjunto daba error, pero 1 a 1 funcionaba bien, por esto se implementó el loop

In [ ]:
humedales_.plot()

***
# Realizar compuestos 

Agruparemos por tiempo calculando la mediana anual de los meses de diciembre, enero, febrero y marzo.

Primero, armamos un vector con las fechas de interés que están en el cubo, y generamos un subconjunto.


In [ ]:
summer_dates = dsf.time[dsf.time.dt.month.isin([12, 1, 2, 3])]
ds_summer = dsf.sel(time=summer_dates)

Es necesario generar una variable por temporada. Diciembre (12), pertenece al año anterior, por lo que no se puede utilizar el año directamente. 

Para solucionarlo, tomaremos la coordenada de base `time` y le añadimos 31 días (aquí nos interesa el año, no el mes, por lo tanto no hay problema al hacerlo de esta forma).

In [ ]:
ds_summer = ds_summer.assign_coords(season=(ds_summer.time + np.timedelta64(31, 'D')))
ds_summer

La coordenada ha sido añadida satisfactoriamente, por lo que ahora se puede usar como variable de agrupamiento y calcular la mediana de la temporada (ignorando valores ausentes)

In [ ]:
# ds_summer = ds_summer.groupby('season.year').reduce(np.nanmean)  # más lento: favorecer las funciones de xarray
ds_summer_ag = ds_summer.groupby('season.year').median(skipna=True).persist()
# ds_summer = ds_summer.groupby('season.year').mean(skipna=True)
ds_summer_ag

La serie temporal agregada está lista, podemos visualizarla para el 2014 (en color y el índice NDVI):

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)

ds_summer_ag[["red", "green", "blue"]].sel(year=2014).to_array().plot.imshow(vmin=0,vmax=.3,ax=ax1) #figsize=(10,10)
ax1.set_xlabel('Easting (m)', fontweight='bold')
ax1.set_ylabel('Northing (m)', fontweight='bold')

ax2 = fig.add_subplot(122)
ds_summer_ag['ndvi'].sel(year=2014).plot.imshow(cmap="RdYlGn", ax=ax2)

Si enmascaramos la zona particular del estudio, utilizando el buffer de humedales, el resultado se ve más limpio:

In [ ]:
import rasterio as rio

smask = rio.features.geometry_mask(humedales_.geometry, 
                                   out_shape=(len(ds.y), len(ds.x)),
                                   transform=ds.geobox.transform,
                                   invert=True)

ds_summer_masked = ds_summer_ag.where(smask)


fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)

ds_summer_masked[["red", "green", "blue"]].sel(year=2014).to_array().plot.imshow(vmin=0,vmax=.3,ax=ax1) #figsize=(10,10)
ax1.set_xlabel('Easting (m)', fontweight='bold')
ax1.set_ylabel('Northing (m)', fontweight='bold')

ax2 = fig.add_subplot(122)
ds_summer_masked['ndvi'].sel(year=2014).plot.imshow(cmap="RdYlGn", ax=ax2)

Además, podemos sobreponer los contornos de los humedales delimitados:

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)

ds_summer_ag[["red", "green", "blue"]].sel(year=2014).to_array().plot.imshow(vmin=0,vmax=.3,ax=ax1) #figsize=(10,10)
humedales_or.plot(ax=ax1, facecolor='none', edgecolor='red')
ax1.set_xlabel('Easting (m)', fontweight='bold')
ax1.set_ylabel('Northing (m)', fontweight='bold')

ax2 = fig.add_subplot(122)
ds_summer_masked['ndvi'].sel(year=2014).plot.imshow(cmap="RdYlGn", ax=ax2)
humedales_or.plot(ax=ax2, facecolor='none', edgecolor='black')

Finalmente, la información puede ser exportada como netcdf, para ser utilizada en otras plataformas/programas

In [ ]:
from datacube.drivers.netcdf import write_dataset_to_netcdf
import warnings
warnings.filterwarnings('ignore')

if os.path.exists('humedales.nc'):
    os.remove('humedales.nc')
    
write_dataset_to_netcdf(ds_summer_masked, 'humedales.nc')

***

# Obtener estadísticas zonales

Extraemos las estadísticas para cada zona iterando por cada año. Debemos procurar que tanto el archivo vectorial como el xarray tengan el mismo CRS.

In [ ]:
my_stats = []
for i, date in enumerate(ds_summer_ag.year.values):
    if i % 5 == 0:
        print('{} - {}'.format(i, date))
    temp = rs.zonal_stats(humedales_, 
                          ds_summer_ag.ndvi.isel(year=i).values, 
                          affine=ndvi.affine, 
                          stats="count min mean max median std", 
                          nodata=np.nan) # especificar no data, solo para evitar advertencias. Los valores NA ya están como np.nan
    temp_ = pd.merge(pd.DataFrame(humedales_.reset_index().drop(columns="geometry")), 
                             pd.DataFrame(temp), 
                             left_index=True, 
                             right_index=True)
    temp_['Año'] = date
    my_stats.append(temp_)

***

#  Visualizar resultados

Observemos la variación de la mediana de todos los humedales del área de estudio:

In [ ]:
serie_temporal = pd.concat(my_stats)
pd.pivot(serie_temporal, index="Año", columns='Id_humedal', values="median").plot(figsize=(12,12))

Esta es la ubicación de cada humedal:

In [ ]:
ax = humedales_or.plot(figsize=(15, 15))
humedales_or.apply(lambda x: ax.annotate(text=x.Id_humedal, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);

Los humedales de interés son:

In [ ]:
humedales_id = ['HU-OT-4023', # humedal objetivo
               'HU-OT-4028', 'HU-OT-4029', 'HU-OT-4030', 'HU-OT-4031', 'HU-OT-4033', 'HU-OT-4040'] # humedales control

Utilizando solo el humedal objetivo visualicemos sus estadísticas en el tiempo:

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

agg_ = serie_temporal.loc[serie_temporal.Id_humedal == humedales_id[0]].set_index('Año')[['min', 'mean', 'median', 'max']]

agg_.plot(ax = ax1) #, subplots=True)
agg_[['min', 'mean', 'median']].plot(ax = ax2) #, subplots=True)

Comparemos el humedal objetivo con los de control (en su media y viendo su tendencia general):

In [ ]:
compara = serie_temporal.loc[serie_temporal.Id_humedal.isin(humedales_id)][['Año', 'Id_humedal', 'mean']]
pd.pivot(compara, index="Año", columns='Id_humedal', values="mean").plot(figsize=(12,12))

In [ ]:
cmap = plt.cm.tab10
colores = cmap(np.linspace(0, 1, len(humedales_id)))
colores = {humedales_id[i]: colores[i] for i in range(len(humedales_id))}

fig, ax = plt.subplots(figsize=(15, 15))

for i in humedales_id:
    temp = serie_temporal.loc[serie_temporal.Id_humedal == i]
    x, y = temp[['Año', 'mean']].T.to_numpy()
    plt.plot(x, y, '-o', color=colores[i])
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x, p(x), "--", color=colores[i])

Los resultados son bastante similares a los obtenidos por el Ministerio del Medio Ambiental, y se aprecia claramente que el humedal objetivo (HU-OT-4023, en azul), tiene un marcado descenso que no está presente en el resto de los humedales

In [ ]:
client.close()

cluster.close()

***

# *Siguientes pasos* &#128062;

Para continuar con el tutorial pueden acceder a los notebooks del siguiente listado.

1. [Acceso](00_Acceso.ipynb)
2. [Cargar datos](01_Cargar_datos.ipynb)
3. [Limpieza](02_Limpieza.ipynb)
4. [Análisis básico](03_Análisis_básico.ipynb)
5. [Caso de estudio 1](04_Caso_de_estudio_1.ipynb)
6. **Caso de estudio 2**

***